In [ ]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
!mkdir .kaggle
!mv kaggle.json .kaggle/
!mv .kaggle ~/

# Proyecto 1: Clasificación de Imagenes

De la página [kaggle](https://www.kaggle.com/), descarguen una dataset de clasificación de imagenes. Por ejemplo:
- [dogs and cats dataset](https://www.kaggle.com/chetankv/dogs-cats-images).
- [flowers](https://www.kaggle.com/alxmamaev/flowers-recognition) 
- [pokemon](https://www.kaggle.com/lantian773030/pokemonclassification)
- [American Sign Language](https://www.kaggle.com/grassknoted/asl-alphabet)
- [Rock, Paper, Scissors](https://www.kaggle.com/drgfreeman/rockpaperscissors)

Pueden usar otras plataformas para buscar y descargar los datastes (como google dataset), o tambien pueden crear su propio dataset de las cosas que quisieran classificar (alrededor de 20~50 imagenes por clases estaria bien).

Es necesario que la carpeta que contiene a las imagenes tenga el siguiente formato:
```
dataset/
 clase1/
  puede_haber_subcarpetas/
    imagen1.jpg
    imgen2.jpg
 clase2/
  solo_las_imagenes.jpg
 clase3/
 ....
```

## Reporte

Diseñen una red neuronal para que clasifique las imagenes que se descargaron.

**Explicar el tipo de dataset que se esta utilizando**

In [ ]:
# TODO: Modificar esto acorde al dataset que se va a usar
!kaggle datasets download chetankv/dogs-cats-images
!unzip dogs-cats-images.zip

**Si se modifica algo en la celda inferior explicar que cambios se hizo**

In [ ]:
def evaluate(model, loader, crit):
  model.eval()
  total = 0
  corrects = 0
  avg_loss = 0
  for x, y in loader:
    x = x.to(dev)
    y = y.to(dev)
    o = model(x)
    loss = crit(o,y)
    avg_loss += loss.item()
    corrects += torch.sum(torch.argmax(o,axis=1) == y).item()
    total += len(y)
  acc = 100* corrects / total
  avg_loss /= len(loader)  
  return avg_loss, acc

def train_one_epoch(model, train_loader, crit, optim):
  model.train()
  total = 0
  corrects = 0
  avg_loss = 0
  for x, y in train_loader:
    optim.zero_grad()
    x = x.to(dev)
    y = y.to(dev)
    o = model(x)
    loss = crit(o,y)
    avg_loss += loss.item()
    loss.backward()
    optim.step()
    corrects += torch.sum(torch.argmax(o,axis=1) == y).item()
    total += len(y)
  acc = 100 * corrects / total
  avg_loss /= len(train_loader)
  return avg_loss, acc

def train(model, train_loader, test_loader, crit, optim, epochs = 20):
  for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader,crit, optim)
    test_loss, test_acc = evaluate(model, test_loader, crit)
    print(f"epoch: {epoch}, train loss: {train_loss}, train acc: {train_acc}%, test loss: {test_loss}, test acc: {test_acc}%")

**Si se modifica algo en la celda inferior explicar que cambios se hizo**

In [ ]:
img_transform = torchvision.transforms.Compose([
  torchvision.transforms.RandomRotation(18),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.Resize((224,224)),
  torchvision.transforms.ToTensor()
])

In [ ]:
train_ds = torchvision.datasets.ImageFolder("./dataset/training_set",transform=img_transform)
test_ds = torchvision.datasets.ImageFolder("./dataset/test_set",transform=img_transform)

In [ ]:
# Esto es solo para poder ver un ejemplo de las imagenes que se estan usando
plt.imshow(train_ds[10][0].numpy().transpose(1,2,0))
plt.show()

**Si se modifica algo en la celda inferior explicar que cambios se hizo**

No se modificó nada

In [ ]:
# Esto no se necesita modificar al menos que se quiera utilizar un batch size diferente
# OPTIONAL:
# Cambiar la opción de shuffle a False y observar que pasa con los input y targets que nos brinda el dataloader, que diferencias hay?
# Observar que pasa con el accuracy cuando no se realiza el shuffling y explicar porque.

train_dl = torch.utils.data.DataLoader(train_ds,batch_size=128,shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds,batch_size=128,shuffle=True)

**Defina su modelo**

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,16,7,bias=False),
    nn.BatchNorm2d(16),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2),
    nn.Conv2d(16,32,3,bias=False),
    nn.BatchNorm2d(32),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2),
    nn.Conv2d(32,32,3,bias=False),
    nn.BatchNorm2d(32),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2),
    nn.Conv2d(32,64,3,bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2),
    nn.Conv2d(64,64,3,bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(1024,2)
).to(dev)

**Si se modifica algo en la celda inferior explicar que cambios se hizo**

In [ ]:
crit = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(),lr=0.1)
train(model,train_dl, test_dl, crit, optim, epochs=10)

In [ ]:
model.eval()
# idx = 10
idx = 1000
x, y = test_ds[idx]
x_numpy = x.numpy().transpose(1,2,0)
N, H, W = x.shape
x = x.reshape(1,N,H,W)
pred = torch.argmax(model(x.to(dev)).cpu()).item()
print(pred)
plt.imshow(x_numpy)

In [ ]:
model.eval()
# idx = 10
for idx in (153,239,3,900,48,1178,456):
  x, y = test_ds[idx]
  x_numpy = x.numpy().transpose(1,2,0)
  N, H, W = x.shape
  x = x.reshape(1,N,H,W)
  pred = torch.argmax(model(x.to(dev)).cpu()).item()
  print(pred)
  plt.imshow(x_numpy)
  plt.show()

In [ ]:
torch.save(model.state_dict(),"proyecto1.ckpt")